# Foldable Words

by Brian Hayes

The code provided here accompanies an article, “Foldable Words,” published in February 2021 on [bit-player](http://bit-player.org/2021/foldable-words). The code is released under an MIT license.

The aim is to answer this question: Given a source text—a string of letters—what words and phrases can be formed by deleting selected letters, and then closing up the spaces between the remaining characters. We call this operating _folding_, since it can be accomplished by making folds in a strip of paper so that the deleted letters are tucked away out of sight. For example, the text “It’s a pleasure to serve you” can be folded to say “I love you.”

Throughout this discussion we ignore all whitespace, punctuation, and capitalization in the text. 

In [ ]:
# for more compact output of lists

%pprint   # Pretty printing is on by default in Jupyter notebooks. Execute
          # this cell to turn it off. The command is a toggle: running it
          # a second time will turn pretty printing on again.

In [ ]:
import re       # regular expressions

import random   # for random sampling

from IPython.core.display import display, HTML   # for formatting output

### Word Lists

You need at least one list of words to do anything useful with these programs. I recommend some version of the lists used by Scrabble players, such as Collins Scrabble Words or SOWPODS. Distribution of these lists is restricted by copyright, but they are available nonetheless.

One alternative to the Scrabble lists is the Unix word list, preinstalled on most Unix or Unix-derived systems.

If you have installed the Natural Language Toolkit, you can access a word list at nltk.corpus.words.words(). (This list is identical to the Unix words list with six additions.)

Other lists (including some Scrabble lists) are available through Peter Norvig's website https://norvig.com/ngrams/.

Whatever the source of the list, it should consist of plain text with one word per line.

In [ ]:
# Load a word list. Replace the argument 'words.txt' with the path to the file.

wordfile = open('words.txt')
lexicon = wordfile.read().splitlines()
lexicon = [w.lower() for w in lexicon]     # convert any Caps to lower case
lexicon = sorted(list(set(lexicon)))       # de-duplicate, then sort
len(lexicon)                               # return length of the list

In [ ]:
# Strip the source text of any characters outside the set [a-z].
# Example: "It's a pleasure to serve you." ==> "itsapleasuretoserveyou"

def normalize(text):
    text = text.lower()
    return ''.join(re.findall(r'[a-z]', text))

### Preliminary experiments

Take random samples of foldable words, and generate all words
of a fixed length.

In [ ]:
# Generate a random sample of foldable words. Offers a quick-and-dirty
# view of typical results.
#
# Args: text:    the source text, a string
#       lexicon: the word list
#       k:       length of the words to be generated, integer
#       reps:    number of random trials
#
# The basic idea is to choose k distinct integers in range(0, n-1),
# where n is the length of the normalized source text. We then sort
# these numbers in ascending order, and use them as indices into the
# source text. The letters chosen in this way are assembled into a 
# string. If that string is present in the lexicon, append it to the
# list of results.

def randomFoldableWords(text, lexicon, k, reps):
    normtext = normalize(text)             # strip out punctuation etc.
    n = len(normtext)
    findings = []
    for i in range(reps):
        indices = random.sample(range(n), k) # k distinct values from {0..n-1}
        indices.sort()
        letters = ""
        for idx in indices:
            letters += normtext[idx]
        if letters in lexicon:
            findings.append(letters)
    return findings

In [ ]:
# Generate all foldable words of exactly three letters.
# A model of how to do it, but it doesn't scale well to
# longer words.


def foldableStrings3(lexicon, text):
    normtext = normalize(text)
    n = len(normtext)
    words = []
    for i in range(0, n-2):
        for j in range(i+1, n-1):
            for k in range(j+1, n):
                s = normtext[i] + normtext[j] + normtext[k]
                if s in lexicon:
                    words.append(s)
    return(words)

### Generate All Foldable Substrings by Counting

The set of all foldable strings from a text of $N$ characters has a one-to-one mapping onto the set of $N$-bit binary numerals. Thus we can generate the strings by counting from 0 to $2^N - 1$, and choosing letters that correspond to the 1 bits in the binary representation of each number. (This idea works, but it's inefficient, since very few of the generated strings turn out to be English words.)

In [ ]:
# Given a number n, return an array listing the digit positions where
# the binary representation on n has a 1 bit. The binary numeral is
# zero-padded on the left to width bits. 

def positionsOf1Bits(n, width):
    cursor = 0
    positions = []
    for i in range(width):
        if n & 1 == 1:               # bitwise AND with LSB
            positions.append(cursor)
        cursor += 1
        n = n >> 1                   # shift right by one bit
    return(positions)

In [ ]:
# Use the binary counting method to generate all possible ordered
# substrings of the source text, then search for each such string
# in the lexicon.


def foldablesByCounting(lexicon, text):
    normtext = normalize(text)
    n = len(normtext)
    words = []
    for i in range(2**n - 1):
        charSeq = ''
        positions = positionsOf1Bits(i, n)
        for p in positions:
            charSeq += normtext[p]
        if charSeq in lexicon:
            words.append(charSeq)
    return(words)

### The Inverse Method: Attempt to Fold Each Known Word

Instead of generating $2^N$ foldable strings and testing each one against the lexicon, take each word in the lexicon and test to see if it can be folded from the source text. Since the lexicon is smaller than $2^N$ for interesting values of $N$, this procedure should be more efficient. It is made even faster by prefiltering the lexicon to remove words that include letters not in the source text.

In [ ]:
# Predicate: Return True iff all characters in word w are
#            present in character set cset. (We needn't
#            consider w if this test fails.)

def wordCharset(w, cset):
    for c in w:
        if c not in cset:
            return(False)
    return(True)

In [ ]:
# Predicate: Return True iff word can be formed by folding the given text.

def wordIsFoldable(word, text):
    normtext = normalize(text)
    t = 0                      # pointer to positions in normtext
    w = 0                      # pointer to positions in word
    while t < len(normtext):
        if word[w] == normtext[t]:  # matching chars in word and text
            w += 1                  # move to next char in word
        if w == len(word):          # we have matched all chars in word
            return(True)            # so: thumbs up
        t += 1                 # move to next char in text
    return(False)              # fell off the end of the text: thumbs down

In [ ]:
# Simple sequential-search implementation. Returns a list
# of all foldable words in text. (The list will be alphabetized
# if the lexicon is.)

# NOTE: This procedure returns a list of foldable words without
# duplicates. Some words can be formed in more than one way, but
# that fact is not signalled here.

def searchFoldableWords_Sequential(lexicon, text):
    normtext = normalize(text)
    cset = set(normtext)                                               # the characters appearing in normtext
    candidates = [w for w in lexicon if wordCharset(w, cset)]          # keep only words formed entirely from chars in cset
    foldables = [w for w in candidates if wordIsFoldable(w, normtext)] # list all candidates that pass the foldability test
    return(foldables)

### Indexed Inverse Method

The searchFoldableWords_Sequential procedure spends most of its time making many repeated sequential searches through the text, looking for the same characters over and over. We can speed things up by precomputing the positions of all the letters in the text, then using this index to go directly to the correct position for each successive character in a word.

This procedure also allows us to detect multiple foldings that produce the same words, and to sort the word instances into groups according to their position in the text.

In [ ]:
# Given a character and a normalized text, return a list of all
# the positions in the text where the character appears. For example,
# findLocations('s', "itsapleasuretoserveyou") returns [2, 8, 14]

def findLocations(char, normtext):
    locations = []
    for i in range(len(normtext)):
        if normtext[i] == char:
            locations.append(i)
    return locations

In [ ]:
# Creates and returns an index to character positions in the given
# text. The index is a Python dictionary whose keys are the characters
# present in the text; the value associated with each key is the 
# corresponding list of character positions. For example,
# makeTextIndex("Mississippi") returns:
#   {'i': [1, 4, 7, 10], 'p': [8, 9], 'm': [0], 's': [2, 3, 5, 6]}
# (The order of the entries is unspecified in a Python dictionary.)

def makeTextIndex(text):
    normtext = normalize(text)
    charset = set(normtext)
    index = {}
    for c in charset:
        index[c] = findLocations(c, normtext)
    return index

In [ ]:
# Using a precomputed index into the source text, attempt
# to fold the text to produce the given word w. The start
# parameter tells us where in the text to begin. The procedure
# is greedy: It will find the first or shortest instance of the 
# word following the starting point.

# The returned value is a list of indices into the text, giving
# the positions of each letter of the word w. If the word is
# not foldable, return False.

# Example: tryToFoldWord_Indexed("see", makeTextIndex("It's a pleasure to serve you"), 0)  ==> [2, 6, 11]
#          tryToFoldWord_Indexed("see", makeTextIndex("It's a pleasure to serve you"), 12) ==> [14, 15, 18]

def tryToFoldWord_Indexed(w, index, start):
    cursor = start                   # pointer into the text
    seqOfIndices = []
    for c in w:
        locations = index[c]         # list of locations where char c appears
        for loc in locations:
            if loc >= cursor:        # exclude locations we have already passed by
                seqOfIndices.append(loc)
                cursor = loc + 1     # move to next position in text
                break                # back to the outer loop for next c in w
    if len(seqOfIndices) < len(w):
        return False                 # failed to fold
    else:
        return seqOfIndices       

In [ ]:
# Now we loop through all the candidate words (those with the right character set),
# and try to fold each one, using the index. The words found are grouped according
# to their starting position in the text. The wantHTML parameter determines whether
# or not to format the output.

# Multiple instances of the same word are recorded if the instances are disjoint, or
# nonoverlapping. See note below for more on this issue.

# The raw output is a list of lists, one inner list for each character position
# in the text. Each of these inner lists includes all the words that begin at
# that position. The elements of the inner lists are tuples made up of the
# foldable word itself and an integer representing its ending position.

# Example: searchFoldableWords_Indexed(lexicon, "straw ", False) yields
# this raw output:
# [[('saw', 4), ('st', 1), ('staw', 4), ('straw', 4)], [('ta', 3), ('taw', 4)], [('raw', 4)], [('a', 3), ('aw', 4)], []]

# For comments on HTML formatted output, see below.

def searchFoldableWords_Indexed(lexicon, text, wantHTML=True):
    normtext = normalize(text)
    cset = set(normtext)
    ix = makeTextIndex(normtext)
    candidates = [w for w in lexicon if wordCharset(w, cset)]
    
    foldables = []                                                     # create a list of empty lists,
    for position in range(len(normtext)):                              # one for each character position
        foldables.append([])                                           # in the text
        
    for w in candidates:                                                  # loop through all candidate words
        for startingPoint in ix[w[0]]:                                    # start with first occurrence of word's first letter
            seqOfIndices = tryToFoldWord_Indexed(w, ix, startingPoint)    # go try to fold the word from this starting point
            if seqOfIndices:                                              # if successful...
                foldables[seqOfIndices[0]].append((w, seqOfIndices[-1]))  # take the inner list indexed by the position of the
                                                                          #    word's first letter, and append a tuple formed
                                                                          #    from the word itself and the position of its
                                                                          #    last letter. ([-1] designates the last element
                                                                          #    of a list.)
    
    if wantHTML:                                               # if formatting is wanted, call formatGroupLists;
        return(HTML(" ".join(formatGroupLists(foldables))))
    else:                                                      # otherwise return the raw list of lists of tuples
        return(foldables)

### A Note on Multiple Word Instances

"It's a pleasure to serve you," can be folded in six ways to form the word "see":

1. <code><span style="color: #aaa">it</span><strong>s</strong><span style="color: #aaa">apl</span><strong>e</strong><span style="color: #aaa">asur</span><strong>e</strong><span style="color: #aaa">toserveyou</span></code>

2. <code><span style="color: #aaa">it</span><strong>s</strong><span style="color: #aaa">apl</span><strong>e</strong><span style="color: #aaa">asure</span><span style="color: #aaa">tos</span><strong>e</strong><span style="color: #aaa">rveyou</span></code>

3. <code><span style="color: #aaa">it</span><strong>s</strong><span style="color: #aaa">apl</span><strong>e</strong><span style="color: #aaa">asure</span><span style="color: #aaa">toserv</span><strong>e</strong><span style="color: #aaa">you</span></code>

4. <code><span style="color: #aaa">itsaplea</span><strong>s</strong><span style="color: #aaa">ur</span><strong>e</strong><span style="color: #aaa">tos</span><strong>e</strong><span style="color: #aaa">rveyou</span></code>

5. <code><span style="color: #aaa">itsaplea</span><strong>s</strong><span style="color: #aaa">ur</span><strong>e</strong><span style="color: #aaa">toserv</span><strong>e</strong><span style="color: #aaa">you</span></code>

6. <code><span style="color: #aaa">itsapleasureto</span><strong>se</strong><span style="color: #aaa">rv</span><strong>e</strong><span style="color: #aaa">you</span></code>

The procedure presented here finds instances 1, 4, and 6, but not 2, 3, and 5. I maintain this is correct behavior, given the aim of combining the foldable words into phrases or sentences. There may be occasions to favor a word instance that falls toward the beginning or the end of the text, or in the middle. But there is never a reason to favor a less-compact instance—one that covers a larger span of the text, and thereby precludes the use of more letters in other folded words.

In [ ]:
# Convert the raw output of searchFoldableWords_Indexed to a more readable HTML format.

# Each list of foldable words is sorted in ascending order of the word's final letter, 
# which is given as the second element of each tuple, i.e., tp[1]. Then each tuple is
# converted to a string consisting of the word followed by a subscript numeral, representing
# the final character index. We then prepend a group number to each list, wrap the list
# in paragraph tags, and assemble it into one big string, with the words separated by
# spaces.


def formatGroupLists(foldables):
    for i in range(len(foldables)):
        foldables[i].sort(key = lambda tp: tp[1])
        foldables[i] = list(map(lambda tp: tp[0] + "<sub>" + str(tp[1]) + "</sub>", foldables[i]))
        foldables[i].insert(0, "<p class='undent' style='margin-bottom: 6pt;'><b>Group " + str(i) + ":</b>")
        foldables[i].append("</p>")
        foldables[i] = " ".join(foldables[i])
    return(foldables)

### Sample Outputs

The examples below were all generated with a word list based on the 2015 edition of Collins Scrabble Words, to which I have added the three one-letter words _a_, _I,_ and _O_. Other word lists will give somewhat different results. 

In [19]:
# Randomized procedure: Expect every run to produce different output.

randomFoldableWords("It's a please to serve you.", lexicon, 6, 10000)

['leaser', 'paster', 'slatey', 'eatery', 'saeter', 'taster', 'sleets', 'storey', 'salses', 'pastor', 'plaste', 'plaste', 'paster', 'tapets', 'islets', 'slater', 'pastor', 'stereo', 'setose', 'setose', 'slater', 'seater', 'slatey', 'paseos']

In [20]:
foldableStrings3(lexicon, "It's a pleasure to serve you.")

['its', 'ita', 'ita', 'its', 'its', 'iso', 'iso', 'iso', 'iso', 'ire', 'ire', 'ire', 'its', 'ios', 'iso', 'ire', 'ivy', 'tap', 'tae', 'tas', 'tau', 'tar', 'tae', 'tat', 'tao', 'tas', 'tae', 'tar', 'tav', 'tae', 'tay', 'tao', 'tau', 'tea', 'tes', 'tee', 'tet', 'tes', 'tee', 'tee', 'tas', 'tau', 'tar', 'tae', 'tat', 'tao', 'tas', 'tae', 'tar', 'tav', 'tae', 'tay', 'tao', 'tau', 'tut', 'try', 'tet', 'tes', 'tee', 'tee', 'toe', 'tor', 'toe', 'toy', 'too', 'tee', 'try', 'sap', 'sal', 'sae', 'sau', 'sar', 'sae', 'sat', 'sae', 'sar', 'sav', 'sae', 'say', 'sau', 'spa', 'spy', 'sly', 'sea', 'ser', 'see', 'set', 'see', 'ser', 'see', 'sey', 'sau', 'sar', 'sae', 'sat', 'sae', 'sar', 'sav', 'sae', 'say', 'sau', 'sur', 'sue', 'sus', 'sue', 'sur', 'sue', 'set', 'see', 'ser', 'see', 'sey', 'sty', 'sos', 'sov', 'soy', 'sou', 'ser', 'see', 'sey', 'sey', 'sou', 'ape', 'ape', 'apt', 'apo', 'ape', 'ape', 'apo', 'ale', 'ala', 'als', 'alu', 'ale', 'alt', 'als', 'ale', 'ale', 'alu', 'aas', 'aas', 'ass', 'aue'

In [21]:
# Shortened text to avoid excessive running time.

foldablesByCounting(lexicon, "It's a pleasure.")

['i', 'it', 'is', 'its', 'a', 'ta', 'ita', 'tap', 'sap', 'al', 'sal', 'te', 'ae', 'tae', 'sae', 'pe', 'ape', 'tape', 'isle', 'ale', 'tale', 'sale', 'a', 'ta', 'ita', 'aa', 'pa', 'spa', 'tapa', 'la', 'ala', 'tala', 'ea', 'tea', 'sea', 'pea', 'lea', 'ilea', 'talea', 'plea', 'is', 'its', 'as', 'tas', 'itas', 'taps', 'saps', 'als', 'sals', 'es', 'tes', 'taes', 'pes', 'apes', 'tapes', 'les', 'isles', 'ales', 'tales', 'sales', 'as', 'tas', 'itas', 'aas', 'pas', 'spas', 'tapas', 'las', 'alas', 'talas', 'eas', 'teas', 'seas', 'peas', 'leas', 'pleas', 'tau', 'sau', 'tapu', 'alu', 'plu', 'leu', 'tau', 'sau', 'eau', 'ar', 'tar', 'sar', 'tsar', 'er', 'ser', 'per', 'aper', 'taper', 'taler', 'ar', 'tar', 'sar', 'tsar', 'par', 'spar', 'lar', 'alar', 'talar', 'ear', 'tear', 'sear', 'pear', 'spear', 'lear', 'ur', 'sur', 'pur', 'spur', 'lur', 'slur', 'sur', 'te', 'ae', 'tae', 'sae', 'pe', 'ape', 'tape', 'isle', 'ale', 'tale', 'sale', 'ee', 'tee', 'see', 'pee', 'lee', 'slee', 'alee', 'ae', 'tae', 'sae', 

In [24]:
searchFoldableWords_Sequential(lexicon, "It's a pleasure to serve you.")

['a', 'aa', 'aas', 'ae', 'aero', 'aeros', 'aery', 'al', 'ala', 'alae', 'alar', 'alary', 'alas', 'alastor', 'alate', 'alay', 'ale', 'aleatory', 'alee', 'alert', 'alerter', 'alerts', 'ales', 'aloe', 'aloo', 'als', 'also', 'alt', 'alter', 'alto', 'altos', 'alts', 'alu', 'alure', 'alures', 'alus', 'apart', 'apay', 'ape', 'aper', 'apers', 'apert', 'apery', 'apes', 'apo', 'apos', 'apres', 'apse', 'apses', 'apso', 'apsos', 'apt', 'apter', 'apts', 'ar', 'are', 'arere', 'ares', 'aret', 'arete', 'arets', 'arose', 'ars', 'arse', 'arsey', 'arsy', 'art', 'artery', 'arts', 'artsy', 'arty', 'arvo', 'ary', 'as', 'ass', 'aster', 'at', 'ate', 'ats', 'aue', 'aures', 'auto', 'autos', 'ave', 'avo', 'ay', 'ayu', 'ea', 'ear', 'ears', 'eas', 'ease', 'easer', 'eases', 'east', 'easter', 'easts', 'easy', 'eat', 'eater', 'eatery', 'eats', 'eau', 'eaus', 'eave', 'ee', 'eery', 'er', 'ere', 'eres', 'erev', 'eros', 'erose', 'err', 'ers', 'es', 'eses', 'ess', 'esse', 'est', 'ester', 'estro', 'ests', 'et', 'euro', 'eur

In [25]:
searchFoldableWords_Indexed(lexicon, "It's a pleasure to serve you.")